# Introduction | Anime face dataset

The goal is to create a neural model (DCGAN) capable of generating new anime face images based on the Kaggle dataset. 

## Importing libraries, funcs & data


In [70]:
import os
import numpy as np
import warnings
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, array_to_img
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")


## Importing dataset 

In [71]:
BASE_DIR = r'/kaggle/input/animefacedataset/images'

In [72]:
image_paths = []
for img in os.listdir(BASE_DIR):
  img_path = os.path.join(BASE_DIR, img)
  image_paths.append(img_path)

In [73]:
image_paths =image_paths[:int(len(image_paths)*0.5)]
len(image_paths)

In [74]:
image_paths[0]

# Preprocessing  data

In [75]:
plt.figure(figsize =(20,20))
temp_images = image_paths[:49]
index  = 1
for image_path in temp_images:
  plt.subplot(7,7,index)
  index += 1
  img = load_img(image_path)
  img = array_to_img(img)
  plt.imshow(img)
  plt.axis('off')

In [76]:

height = width =  64
target_size = (height, width)

train_images = [img_to_array(load_img(path, target_size=target_size)) for path in tqdm(image_paths)]
train_images = np.array(train_images)


In [77]:
train_images[0].shape

In [78]:
train_images = train_images.reshape(train_images.shape[0],height,width,3).astype('float32')

In [79]:
train_images = (train_images-127.5)/127.5

In [80]:
train_images[0]

## Deep Convolutional Generative Adversarial Network 


### Creating the Generator

In [81]:
LATENT_DIM = 100
WEIGHT_INIT = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
CHANNELS =3

In [82]:
def Generator_Model():
    gen = Sequential(name='generator')
    gen.add(layers.Dense(8*8*512,input_dim=LATENT_DIM))
    gen.add(layers.BatchNormalization())
    gen.add(layers.ReLU())

    gen.add(layers.Reshape((8,8,512)))
    gen.add(layers.Conv2DTranspose(256,(4,4), strides=(2,2),padding='same', kernel_initializer=WEIGHT_INIT))
    gen.add(layers.BatchNormalization())
    gen.add(layers.ReLU())

    gen.add(layers.Conv2DTranspose(128,(4,4), strides=(2,2),padding='same', kernel_initializer=WEIGHT_INIT))
    gen.add(layers.BatchNormalization())
    gen.add(layers.ReLU())

    gen.add(layers.Conv2DTranspose(64,(4,4), strides=(2,2),padding='same', kernel_initializer=WEIGHT_INIT))
    gen.add(layers.BatchNormalization())
    gen.add(layers.ReLU())

    gen.add(layers.Conv2D(CHANNELS, (4,4), padding='same', activation='tanh'))
    return gen 
generator = Generator_Model() 
generator.summary()

### Creating the Discriminator

In [83]:
def Discriminator_Model():
    disc = Sequential(name='discriminator')
    input_shape = (64,64,3)
    alpha = 0.2
    disc.add(layers.Conv2D(64,(4,4), strides=(2,2), padding='same', input_shape=input_shape))
    disc.add(layers.BatchNormalization())
    disc.add(layers.LeakyReLU(alpha=alpha))

    disc.add(layers.Conv2D(128,(4,4), strides=(2,2), padding='same', input_shape=input_shape))
    disc.add(layers.BatchNormalization())
    disc.add(layers.LeakyReLU(alpha=alpha))

    disc.add(layers.Conv2D(128,(4,4), strides=(2,2), padding='same', input_shape=input_shape))
    disc.add(layers.BatchNormalization())
    disc.add(layers.LeakyReLU(alpha=alpha))


    disc.add(layers.Flatten())
    disc.add(layers.Dropout(0.3))

    disc.add(layers.Dense(1, activation='sigmoid'))
    return disc 
discriminator = Discriminator_Model()
discriminator.summary()

### Creating DCGAN

In [84]:
class DCGAN(keras.Model):
  def __init__(self, discriminator, generator, latent_dim):
    super(DCGAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim
    self.g_loss_metric = keras.metrics.Mean(name='g_loss')
    self.d_loss_metric = keras.metrics.Mean(name='d_loss')

  @property
  def metrics(self):
    return [self.g_loss_metric, self.d_loss_metric]

  def compile(self, g_optimizer, d_optimizer, loss_fn):
    super(DCGAN, self).compile()
    self.g_optimizer = g_optimizer
    self.d_optimizer = d_optimizer
    self.loss_fn = loss_fn
    
  def train_step(self, real_images):
    batch_size = tf.shape(real_images)[0]
    random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))
    with tf.GradientTape() as tape:
      pred_real = self.discriminator(real_images, training=True)
      real_labels = tf.ones((batch_size,1))
      d_loss_real = self.loss_fn(real_labels, pred_real)

      fake_images = self.generator(random_noise)
      pred_fake = self.discriminator(fake_images, training=True)
      fake_labels = tf.zeros((batch_size,1))
      d_loss_fake = self.loss_fn(fake_labels, pred_fake)

      d_loss = (d_loss_real+d_loss_fake)/2
    gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)

    self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))

    labels = tf.ones((batch_size, 1))
    with tf.GradientTape() as tape:
      fake_images = self.generator(random_noise)
      pred_fake = self.discriminator(fake_images, training=True)
      g_loss = self.loss_fn(labels, pred_fake)

    gradients = tape.gradient(g_loss, self.generator.trainable_variables)
    self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

    self.d_loss_metric.update_state(d_loss)
    self.g_loss_metric.update_state(g_loss)
    return {
        "g_loss": self.g_loss_metric.result(),
        "d_loss": self.d_loss_metric.result(),
    }

### The monitoring process

In [85]:
class DCGANMonitor(keras.callbacks.Callback):
  def __init__(self, num_img=6, latent_dim=100):
    self.num_img = num_img
    self.latent_dim = latent_dim
    self.noise = tf.random.normal([25, self.latent_dim])

  def on_epoch_end(self, epoch, logs=None):
    g_img = self.model.generator(self.noise)
    g_img = (g_img*127.5)+127.5
    g_img.numpy()
    fig = plt.figure(figsize=(10,10))
    for i in range(self.num_img):
      plt.subplot(5,5,i+1)
      img = array_to_img(g_img[i])
      plt.imshow(img)
      plt.axis('off')
    plt.show()
  def on_train_end(self, logs=None):
    self.model.generator.save('generator.h5')



## Model training

In [86]:
dcgan = DCGAN(generator=generator, discriminator=discriminator, latent_dim=LATENT_DIM)

In [87]:
D_LR = 0.0001
G_LR = 0.0003
dcgan.compile(
    d_optimizer=Adam(learning_rate=D_LR, beta_1=0.5),
    g_optimizer=Adam(learning_rate=G_LR, beta_1=0.5),
    loss_fn=BinaryFocalCrossentropy()
)

In [88]:
N_EPOCHS = 30
history = dcgan.fit(
    train_images,
    epochs=N_EPOCHS,
    callbacks=[DCGANMonitor(num_img=10, latent_dim=LATENT_DIM)]
)

## Model evaluation 

In [46]:
noise = tf.random.normal([1,100])
g_img = dcgan.generator(noise)
g_img = (g_img*127.5)*127.5
g_img.numpy()
img = array_to_img(g_img[0])
plt.imshow(img)
plt.axis('off')
plt.show()


In [103]:
noise = tf.random.normal([1,100])
g_img = dcgan.generator(noise)
g_img = (g_img*127.5)*127.5
g_img.numpy()
img = array_to_img(g_img[0])
plt.imshow(img)
plt.axis('off')
plt.show()


In [98]:
def create_loss_fig(d_loss, g_loss):
    plt.figure(figsize=(10,6))
    plt.plot(d_loss, label='discriminator loss')
    plt.plot(g_loss, label='generator loss')
    plt.title('gen and disc losses')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [92]:
d_loss_values = history.history['d_loss']
g_loss_values = history.history['g_loss']

In [93]:
create_loss_fig(d_loss_values,g_loss_values)